In [1]:
# https://codeleading.com/article/65265993022/
# https://zhuanlan.zhihu.com/p/360854229

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable  # 利用Variable定义一个计算图，可以实现自动求导
from torch.nn import init  # 初始化参数

import numpy as np
import random
import time

from collections import defaultdict  # 替代容器
from sklearn.metrics import f1_score  # 模型、参数的封装

In [2]:
class MeanAggregator(nn.Module) :


    def __init__(self, features, gcn=False, cuda=False) :

        super(MeanAggregator, self).__init__()

        self.features = features  # 2708*1433，，724*128
        self.gcn = gcn  # 是否汇聚本地特征
        self.cuda = cuda  # 是否使用cuda


    def forward(self, nodes, neighs, num_sample=10) :  # 节点集，每个节点的邻居集，采样数量

        # print(nodes)
        # print(len(nodes))  # 256/724
        # print(neighs)
        # print(len(neighs))  # 256/724

        _set = set  # 定义操作，创建空集合
        if not num_sample is None :  # 采样数量非空
            _sample = random.sample  # 定义操作，从指定序列中，不作修改随机截取指定长度的片断
            samp_neighs = [_set(_sample(neigh, num_sample, ))  # 有限采样
                           if len(neigh)>=num_sample else neigh  # 邻居数目可能不到num
                           for neigh in neighs]  # 每个节点的邻居集
        else :
            samp_neighs = neighs

        if self.gcn :
            samp_neighs = [samp_neigh + set([nodes[i]])  # 采样邻居集，加上本地
                           for i, samp_neigh in enumerate(samp_neighs)]  # 摘出index

        # print(samp_neighs)
        # print(len(samp_neighs))  # 256/724


        # 一个batch内的所有节点的邻居节点，去重，聚集，无序
        unique_nodes_list = list(set.union(*samp_neighs))  # *解包，union并集
        unique_nodes = {n:i for i, n in enumerate(unique_nodes_list)}  # 添加索引映射
        # print(unique_nodes_list)
        # print(len(unique_nodes_list))  # 724/1666

        mask = Variable(torch.zeros(len(samp_neighs), len(unique_nodes)))  # 本地节点与采样邻居的邻接矩阵
        row_indices = [i for i in range(len(samp_neighs)) for j in range(len(samp_neighs[i]))]  # i + j
        column_indices = [unique_nodes[n] for samp_neigh in samp_neighs for n in samp_neigh]  # indices vs. indexes
        mask[row_indices, column_indices] = 1
        # print(len(row_indices))  # 922/
        # print(len(column_indices))  # 922/
        # print(mask.shape)  # 256*724/724*1666

        if self.cuda :
            # print("cuda")
            mask = mask.cuda()
        num_neigh = mask.sum(1, keepdim=True)
        # print(num_neigh)
        mask = mask.div(num_neigh)  # normalized
        # print(mask.shape)  # 256*724/724*1666


        # 触发了self.features，程序执行到此处，进行回弹？还是只是显示的问题。
        if self.cuda :
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list).cuda())
        else :
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list))
        # print(len(unique_nodes_list))  # 724？/1666
        # print(embed_matrix.shape)  # 724*1433？/1666*1433/724*128


        neigh_feats = mask.mm(embed_matrix)  # AH: neight_feats = mask * embed_matrix
        # print(neigh_feats.shape)  # 256*1433？/724*1433/256*724 * 724*128 = 256*128

        return neigh_feats

In [3]:
class Encoder(nn.Module) :


    def __init__(self, 

                 adj_lists,  # 邻接列表
                 features,  # 特征矩阵
                 feature_dim,  # 原始特征维度

                 aggregator,  # 汇聚函数
                 embed_dim,  # 映射特征维度

                 base_model=None,  # 基础模型
                 num_sample=10,  # 采样数量
                 gcn=False,  # 是否使用GCN，汇聚本地信息

                 cuda=False,  # 是否使用CUDA
                ) :  # 没有默认值参数的变量，不能放在有默认值参数变量的后面

        super(Encoder, self).__init__()

        self.adj_lists = adj_lists
        self.features = features  # enc2初始化？
        self.feat_dim = feature_dim  # enc2初始化？

        if base_model != None :
            self.base_model = base_model  # 哪里使用？
        self.num_sample = num_sample
        self.gcn = gcn
        self.aggregator = aggregator
        self.embed_dim = embed_dim

        self.cuda = cuda
        self.aggregator.cuda = cuda

        # 执行卷积的参数矩阵。如果不使用GCN模式，需要执行concat拼接操作，所以向量维度为2倍的feat_dim
        self.weight = nn.Parameter(torch.FloatTensor
                                   (embed_dim, self.feat_dim if self.gcn else 2*self.feat_dim))  # 读论文
        init.xavier_uniform(self.weight)  # 参数初始化


    def forward(self, nodes) :

        neigh_feats = self.aggregator.forward(nodes, [self.adj_lists[int(node)] 
                                                      for node in nodes], self.num_sample)
        # enc2-agg2-enc1-agg1  agg1-enc1-agg2-enc2
        # 256*1433？/724*1433/256*128

        if not self.gcn :
            if self.cuda :
                self_feats = self.features(torch.LongTensor(nodes).cuda())
            else :
                self_feats = self.features(torch.LongTensor(nodes))

            combined = torch.cat([self_feats, neigh_feats], dim=1)  # concat
        else :
            combined = neigh_feats

        combined = F.relu(self.weight.mm(combined.t()))
        # 128*1433 * 1433*256 = 128*256 ？
        # 128*1433 * 1433*724 = 128*724
        # 128*128 * 128*256 = 128*256

        return combined

In [4]:
class SupervisedGraphSage(nn.Module) :


    def __init__(self, enc, num_classes) :

        super(SupervisedGraphSage, self).__init__()

        self.enc = enc

        self.weight = nn.Parameter(torch.FloatTensor(num_classes, enc.embed_dim))  # 7*128
        init.xavier_uniform(self.weight)
        self.xent = nn.CrossEntropyLoss()


    def forward(self, nodes) :
        embeds = self.enc(nodes)  # enc2(nodes)
        scores = self.weight.mm(embeds)  # 7*128 * 128*256 = 7*256
        return scores.t()  # 256*7


    def loss(self, nodes, labels) :
        scores = self.forward(nodes)
        return self.xent(scores, labels.squeeze())

In [5]:
def load_cora() :


    num_nodes = 2708
    num_feats = 1433

    feat_data = np.zeros((num_nodes, num_feats))  # 2708*1433
    labels = np.empty((num_nodes, 1), dtype=np.int64)

    node_map = {}  # paper id - node index
    label_map = {}  # class label - label id


    with open("cora/cora.content") as fp :
        # cora.content: paper_id + word_attributes + class_label

        for i, line in enumerate(fp) :
            info = line.strip().split()  # strip()移除字符，split()拆分字符，形成list

            node_map[info[0]] = i  # paper id - node index

            # -1，倒数第一个；数字字符串列表，数字浮点型列表；,为区分，:为切片
            feat_data[i,:] = list(map(float, info[1:-1]))

            if not info[-1] in label_map :
                label_map[info[-1]] = len(label_map)  # 为未知class label赋予label id
            labels[i] = label_map[info[-1]]  # node label id


    adj_lists = defaultdict(set)  # 新建一个以set为默认value的字典
    with open("cora/cora.cites") as fp :
        # cora.cites: ID of cited paper \t ID of citing paper

        for i, line in enumerate(fp) :
            info = line.strip().split()

            paper1 = node_map[info[0]]  # paper id - node index
            paper2 = node_map[info[1]]
            adj_lists[paper1].add(paper2)
            adj_lists[paper2].add(paper1)  # undirected graph


    return adj_lists, feat_data, labels

In [6]:
def run_cora() :


    np.random.seed(1)  # 调用一次，有效一次
    random.seed(1)  # 调用一次，有效一次

    num_nodes = 2708

    adj_lists, feat_data, labels = load_cora()  # 执行一次
    # print(adj_lists)
    # feat_array = np.array(feat_data)
    # print(feat_array.shape)  # 2708*1433

    features = nn.Embedding(2708, 1433)  # 字典查找表，https://zhuanlan.zhihu.com/p/647536930
    features.weight = nn.Parameter(torch.FloatTensor(feat_data), requires_grad=False)
        # 将一个不可训练的tensor转换成可以训练的类型parameter
    # features.cuda()

    agg1 = MeanAggregator(features, cuda=True)  # gcn = False
    enc1 = Encoder(adj_lists, features, 1433, agg1, embed_dim=128, gcn=True, cuda=False)
    enc1.num_samples = 5

    agg2 = MeanAggregator(lambda nodes:enc1(nodes).t(), cuda=False)  # lambda 参数：表达式
    enc2 = Encoder(adj_lists, lambda nodes:enc1(nodes).t(), enc1.embed_dim, 
                   agg2, embed_dim=128, base_model=enc1, gcn=True, cuda=False)
    enc2.num_samples = 5

    graphsage = SupervisedGraphSage(enc2, 7)
    # graphsage.cuda()

    rand_indices = np.random.permutation(num_nodes)  # 随机排列序列
    # 按照节点分集
    train = list(rand_indices[1500:])  # list
    val = rand_indices[1000:1500]
    test = rand_indices[:1000]

    optimizer = torch.optim.SGD(filter(lambda p:p.requires_grad, graphsage.parameters()), lr=0.7)  # 0.7？


    times = []  # 时间戳片
    for batch in range(100) :

        batch_nodes = train[:256]  # 256个点
        random.shuffle(train)  # 每次采样不同

        start_time = time.time()
        optimizer.zero_grad()
        loss = graphsage.loss(batch_nodes, Variable(torch.LongTensor(labels[np.array(batch_nodes)])))  # 真正执行
        loss.backward()
        optimizer.step()
        end_time = time.time()
        times.append(end_time - start_time)

        print(batch, loss.item())  #


    val_output = graphsage.forward(val)
    print("Validation F1:", f1_score(labels[val], val_output.data.numpy().argmax(axis=1), average="micro"))
    print("Average batch time:", np.mean(times))

In [7]:
if __name__ == "__main__" :

    # print(torch.cuda.is_available())

    run_cora()

/tmp/ipykernel_74749/3347669318.py:39: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(self.weight)  # 参数初始化
/tmp/ipykernel_74749/1300380885.py:11: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(self.weight)


0 1.9382734298706055
1 1.9129709005355835
2 1.8952627182006836
3 1.8668767213821411
4 1.8197790384292603
5 1.7842298746109009
6 1.7523261308670044
7 1.728799819946289
8 1.6776642799377441
9 1.6426564455032349
10 1.543238878250122
11 1.533272385597229
12 1.4709399938583374
13 1.4595310688018799
14 1.3914778232574463
15 1.3069311380386353
16 1.2662317752838135
17 1.1955063343048096
18 1.027852177619934
19 0.9776282906532288
20 0.9540759921073914
21 0.9288228750228882
22 0.8156899213790894
23 0.7928718328475952
24 0.7569703459739685
25 0.7816515564918518
26 0.6727970242500305
27 0.6390024423599243
28 0.6474947333335876
29 0.6191361546516418
30 0.5655132532119751
31 0.7349564433097839
32 0.9486786127090454
33 2.2249741554260254
34 0.646219789981842
35 0.5213156342506409
36 0.498539000749588
37 0.48807981610298157
38 0.46968531608581543
39 0.49841806292533875
40 0.501276433467865
41 0.42151889204978943
42 0.37135791778564453
43 0.4268933832645416
44 0.419485867023468
45 0.3656945824623108
4